In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate() # type: ignore

spark

your 131072x1 screen size is bogus. expect trouble
23/04/29 18:26:30 WARN Utils: Your hostname, HOMELAB resolves to a loopback address: 127.0.1.1; using 192.168.38.216 instead (on interface eth0)
23/04/29 18:26:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/29 18:26:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/29 18:26:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    DoubleType,
    DateType
)

scheam = StructType([
    StructField("int_col", IntegerType(), False),
    StructField("float_col", DoubleType(), False),
    StructField("date_col", DateType(), False),
])

df = spark.read.csv("test.csv", header=True, schema=scheam)
df.createOrReplaceTempView("test")
df.show()

+-------+-------------------+----------+
|int_col|          float_col|  date_col|
+-------+-------------------+----------+
|     61| 0.8073818158133667|2020-01-01|
|     35|  0.578711277452551|2020-01-02|
|     15| 0.8364416850373244|2020-01-03|
|     82| 0.9734445815033848|2020-01-04|
|      1| 0.8148832761926326|2020-01-05|
|     65| 0.7622086529648116|2020-01-06|
|      7| 0.5994848115274823|2020-01-07|
|     95| 0.1469505019280003|2020-01-08|
|     73| 0.9432656982951572|2020-01-09|
|     27| 0.8186197361773122|2020-01-10|
|     64|0.18411304478025758|2020-01-11|
|     34|0.15359206987182772|2020-01-12|
|     43| 0.5235404370890783|2020-01-13|
|     37| 0.7013560139096924|2020-01-14|
|      1|0.17739415852070062|2020-01-15|
|     22|  0.929177161270142|2020-01-16|
|     27| 0.8650363274862403|2020-01-17|
|     25|0.26861230228755884|2020-01-18|
|     62| 0.5764202061009532|2020-01-19|
|     95| 0.1838973213370123|2020-01-20|
+-------+-------------------+----------+
only showing top

In [11]:
with open("queries/2.sql", "r") as f:
    query = f.read()
result = spark.sql(query)
print("# rows =", result.count())
result.show()

# rows = 57
+-------+--------------------+----------+
|int_col|           float_col|  date_col|
+-------+--------------------+----------+
|     74| 0.06150215634756562|2020-05-06|
|     25| 0.07493069931545593|2020-05-07|
|     86| 0.03597191951124712|2020-05-11|
|     80|0.005967898083982792|2020-05-12|
|     56|  0.9607835111598791|2020-05-20|
|     48|  0.9032945523700479|2020-05-24|
|     39| 0.03632165245662611|2020-05-30|
|     47| 0.06603691876266982|2020-06-02|
|      6| 0.04239168610926747|2020-06-11|
|     26| 0.08044099615021327|2020-06-12|
|      8| 0.03807827176774192|2020-06-18|
|     72|  0.9641308179892947|2020-06-23|
|     55|  0.9664740012666242|2020-06-27|
|     86| 0.04012405861038182|2020-06-28|
|      4|  0.9432016755817492|2020-06-29|
|     49|  0.9190153099931611|2020-07-02|
|     97|  0.9936241120003995|2020-07-06|
|     24|0.009396728747664396|2020-07-12|
|     20|  0.9049034888850124|2020-07-24|
|     95|0.060510302264966565|2020-07-30|
+-------+-------------

In [142]:
from pglast import parse_sql

with open("queries/1.sql", "r") as f:
    query = f.read()

root = parse_sql(query)
root[0]()

{'@': 'RawStmt',
 'stmt': {'@': 'SelectStmt',
  'distinctClause': None,
  'intoClause': None,
  'targetList': ({'@': 'ResTarget',
    'name': None,
    'indirection': None,
    'val': {'@': 'ColumnRef',
     'fields': ({'@': 'String', 'sval': 'int_col'},),
     'location': 9},
    'location': 9},
   {'@': 'ResTarget',
    'name': None,
    'indirection': None,
    'val': {'@': 'ColumnRef',
     'fields': ({'@': 'String', 'sval': 'float_col'},),
     'location': 20},
    'location': 20},
   {'@': 'ResTarget',
    'name': None,
    'indirection': None,
    'val': {'@': 'ColumnRef',
     'fields': ({'@': 'String', 'sval': 'date_col'},),
     'location': 33},
    'location': 33}),
  'fromClause': ({'@': 'RangeVar',
    'catalogname': None,
    'schemaname': None,
    'relname': 'test',
    'inh': True,
    'relpersistence': 'p',
    'alias': None,
    'location': 49},),
  'whereClause': {'@': 'BoolExpr',
   'boolop': {'#': 'BoolExprType', 'name': 'AND_EXPR', 'value': 0},
   'args': ({'@': 

In [138]:
# from pglast.ast import Node, BoolExpr, A_Expr, SelectStmt
# from pglast.enums import A_Expr_Kind

# exprs = []

# def extract_expr(node):
#     if not isinstance(node, Node):
#         raise TypeError("node must be an instance of Node")      
#     if isinstance(node, BoolExpr):
#         expr = {
#             "op": node.boolop,
#             "args": []
#         }
#         for arg in node.args: # type: ignore
#             res = extract_expr(arg)
#             if res:
#                 expr["args"].append(res)
#         return expr
#     if isinstance(node, A_Expr):
#         if (
#             node.kind == A_Expr_Kind.AEXPR_OP and
#             node.name[0].sval in ["<", ">", "<=", ">="] # type: ignore
#         ):
#             print(node.rexpr)
#             return (node.lexpr.fields[-1].sval, node.name[0].sval, '') # type: ignore
#     else:
#         for member in node:
#             value = getattr(node, member)
#             if isinstance(value, tuple):
#                 values = list(value)
#             else:
#                 values = [value]
#             for sub_node in values:
#                 if isinstance(sub_node, Node):
#                     extract_expr(sub_node)
#         return None
            
# extract_expr(root[0])
# exprs


<SubLink subLinkType=<SubLinkType.EXPR_SUBLINK: 4> subLinkId=0 subselect=<SelectStmt targetList=(<ResTarget val=<FuncCall funcname=(<String sval='avg'>,) args=(<ColumnRef fields=(<String sval='float_col'>,)>,) agg_within_group=False agg_star=False agg_distinct=False func_variadic=False funcformat=<CoercionForm.COERCE_EXPLICIT_CALL: 0>>>,) fromClause=(<RangeVar relname='test' inh=True relpersistence='p'>,) groupDistinct=False limitOption=<LimitOption.LIMIT_OPTION_DEFAULT: 0> op=<SetOperation.SETOP_NONE: 0> all=False>>
<TypeCast arg=<A_Const isnull=False val=<String sval='2020-12-30'>> typeName=<TypeName names=(<String sval='date'>,) setof=False pct_type=False typemod=-1>>


[{'op': <BoolExprType.AND_EXPR: 0>,
  'args': [('float_col', '>', ''), ('date_col', '>', '')]}]